In [1]:
# Inspiration
# Do men or women speed more often?
# Does gender affect who gets searched during a stop?
# During a search, how often is the driver frisked?
# Which year had the least number of stops?
# How does drug activity change by time of day?
# Do most stops occur at night?

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,time, date

In [3]:
lrock = pd.read_csv(r'C:\Users\godwi\OneDrive\Documents\Data Science\GitHub\Python-Pandas\Police\Police\police_project.csv')
lrock.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


In [4]:
lrock.shape

(91741, 15)

In [5]:
lrock.info(verbose = False)  # check what data type is available

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Columns: 15 entries, stop_date to drugs_related_stop
dtypes: bool(2), float64(3), object(10)
memory usage: 9.3+ MB


In [6]:
# Total number of registered male and female drivers
lrock['driver_gender'].value_counts().sum()

86406

# Do men or women speed more often?

In [7]:
# ratio of male to female
lrock['driver_gender'].value_counts().to_frame()

,driver_gender
M,62895
F,23511


In [8]:
# % of male to female
round(lrock['driver_gender'].value_counts(1).to_frame(),3)
# Men speed more than women

,driver_gender
M,0.728
F,0.272


In [9]:
# check if there are any null values in driver_gender column ?
lrock.driver_gender.isnull().value_counts().to_frame()
# so we have 5,335 null values in the drive_gender column 

,driver_gender
False,86406
True,5335


# Does gender affect who gets searched during a stop?

In [10]:
lrock.loc[:, ['driver_gender', 'search_conducted']].groupby('driver_gender')\
.count().rename({'search_conducted' : 'Count_of_search'},axis = 1)

# Yes the male ate searched more than the female

,Count_of_search
driver_gender,
F,23511
M,62895


# During a search, how often is the driver frisked?

In [14]:
arrest = round(lrock.is_arrested.value_counts(100).to_frame(),2)
arrest
# Only 3% of the time , are drivers arrested

,is_arrested
False,0.97
True,0.03


In [15]:
# add a new table , and append %
arrest['%'] = arrest.apply(lambda x : x+)

In [16]:
arrest

,is_arrested,%
False,0.97,1.94
True,0.03,0.06
